<a href="https://colab.research.google.com/github/epignatelli/fenton_karma_jax/blob/master/fenton_karma_jax_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/epignatelli/fenton_karma_jax.git

Cloning into 'fenton_karma_jax'...
remote: Enumerating objects: 183, done.
remote: Counting objects: 100% (183/183), done.
remote: Compressing objects: 100% (161/161), done.
remote: Total 183 (delta 92), reused 32 (delta 12), pack-reused 0
Receiving objects: 100% (183/183), 383.07 KiB | 1.24 MiB/s, done.
Resolving deltas: 100% (92/92), done.


In [ ]:
from fenton_karma_jax import params
from fenton_karma_jax import model
import matplotlib.pyplot as plt
import jax
import jax.numpy as np

In [ ]:
# geometry
shape = (1024, 1024)
dt = 0.01

# diffusivity map
d = 0.05
D = np.ones(shape) * d

# stimuli
field = np.zeros(shape)
field = jax.ops.index_update(field, jax.ops.index[0:20], .8)
s1 = {
    "field": field,
    "start": 0,
    "duration": 2,
    "period": 0  # non-cyclic
}

field2 = np.zeros(shape)
field2 = jax.ops.index_update(field, jax.ops.index[:, -20:], .5)
s2 = {
    "field": field2,
    "start": 200,
    "duration": 2,
    "period": 0  # non-cyclic
}

field3 = np.zeros(shape)
field2 = jax.ops.index_update(field, jax.ops.index[40:60], .5)
s3 = {
    "field": field3,
    "start": 360,
    "duration": 15,
    "period": 0  # non-cyclic
}

In [30]:
%%timeit -n10
state = model.forward(shape, 1000, params.params_test(), D, [s1, s2], dt)[0].block_until_ready()

10 loops, best of 3: 613 ms per loop


In [17]:
%%timeit -n10
state = model.init(shape)
p = params.params_test()
for t in range(1000):
    state = model.step(state, t * dt, p, D, [s1, s2], dt)
state[0].block_until_ready()

10 loops, best of 3: 2.34 s per loop
